In [128]:
import wrangle as wgl
import split_scale as ss
import evaluate as ev
import explore as exp
from env import get_db_url
import pandas as pd
import numpy as np
import seaborn as sns
import feature_engineering as fe

In [4]:
query = '''
select * from properties_2017
'''
df = pd.read_sql(query, get_db_url('zillow'))

#drop object columns
df = df.select_dtypes(exclude=['object'])

In [66]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 56 columns):
id                              int64
parcelid                        int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotal

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,2.985217e+06,2.985217e+06,815362.000000,6061.000000,1627.000000,2.982260e+06,2.982272e+06,12731.000000,1.941395e+06,2.868061e+06,...,2.937384e+06,685676.000000,5163.0,2.938753e+06,2.950951e+06,2.982284e+06,2.925291e+06,2.962465e+06,56517.000000,2.910232e+06
mean,1.492608e+06,1.332586e+07,1.945723,7.202607,647.220652,2.216196e+00,3.093362e+00,3.734742,6.280822e+00,2.303826e+00,...,1.964438e+03,1.400956,1.0,1.781429e+05,4.435279e+05,2.015998e+03,2.684558e+05,5.408949e+03,13.891696,6.048434e+13
std,8.617581e+05,7.909966e+06,3.160507,2.436290,538.787104,1.078250e+00,1.271569e+00,0.500371,1.729923e+00,1.003243e+00,...,2.364428e+01,0.539056,0.0,4.600503e+05,8.163366e+05,6.039949e-02,4.865097e+05,9.675567e+03,2.562172,3.249133e+11
min,0.000000e+00,1.071172e+07,1.000000,2.000000,20.000000,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,...,1.801000e+03,1.000000,1.0,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,2.400000e-01,0.000000,-1.000000e+00
25%,7.463040e+05,1.164371e+07,1.000000,7.000000,272.000000,2.000000e+00,2.000000e+00,3.000000,5.000000e+00,2.000000e+00,...,1.950000e+03,1.000000,1.0,7.766600e+04,1.882200e+05,2.016000e+03,7.970000e+04,2.468620e+03,14.000000,6.037400e+13
50%,1.492608e+06,1.254509e+07,1.000000,7.000000,535.000000,2.000000e+00,3.000000e+00,4.000000,6.000000e+00,2.000000e+00,...,1.963000e+03,1.000000,1.0,1.270660e+05,3.211610e+05,2.016000e+03,1.766190e+05,4.007620e+03,14.000000,6.037572e+13
75%,2.238912e+06,1.409712e+07,1.000000,7.000000,847.500000,3.000000e+00,4.000000e+00,4.000000,8.000000e+00,3.000000e+00,...,1.981000e+03,2.000000,1.0,2.040000e+05,5.140720e+05,2.016000e+03,3.261000e+05,6.230500e+03,15.000000,6.059042e+13
max,2.985216e+06,1.696019e+08,13.000000,27.000000,8516.000000,3.200000e+01,2.500000e+01,5.000000,1.200000e+01,3.200000e+01,...,2.016000e+03,41.000000,1.0,2.553212e+08,3.196225e+08,2.016000e+03,9.401108e+07,3.823176e+06,99.000000,4.830301e+14


In [67]:
#drop id columns
id_columns = []
for i in df.columns.tolist():
    if 'id' in i:
        id_columns += [i]
    
clean_df = df.drop(id_columns, axis = 1)
id_columns

['id',
 'parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'decktypeid',
 'heatingorsystemtypeid',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertylandusetypeid',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'storytypeid',
 'typeconstructiontypeid']

In [68]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 38 columns):
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
calculatedbathnbr               float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                  float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
poolcnt                         float64
poolsizesum                     float64
rawcens

In [69]:
clean_df.describe()

,basementsqft,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,...,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,1627.000000,2.982260e+06,2.982272e+06,2.868061e+06,203758.000000,2.940120e+06,2.720786e+06,7671.000000,190185.000000,203758.000000,...,2.937384e+06,685676.000000,5163.0,2.938753e+06,2.950951e+06,2.982284e+06,2.925291e+06,2.962465e+06,56517.000000,2.910232e+06
mean,647.220652,2.216196e+00,3.093362e+00,2.303826e+00,1379.780460,1.831455e+03,1.764045e+03,1178.916699,2754.869233,1392.029093,...,1.964438e+03,1.400956,1.0,1.781429e+05,4.435279e+05,2.015998e+03,2.684558e+05,5.408949e+03,13.891696,6.048434e+13
std,538.787104,1.078250e+00,1.271569e+00,1.003243e+00,634.422849,1.954198e+03,1.031383e+03,357.093554,5999.380115,682.323851,...,2.364428e+01,0.539056,0.0,4.600503e+05,8.163366e+05,6.039949e-02,4.865097e+05,9.675567e+03,2.562172,3.249133e+11
min,20.000000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,120.000000,112.000000,3.000000,...,1.801000e+03,1.000000,1.0,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,2.400000e-01,0.000000,-1.000000e+00
25%,272.000000,2.000000e+00,2.000000e+00,2.000000e+00,1010.000000,1.215000e+03,1.198000e+03,960.000000,1696.000000,1012.000000,...,1.950000e+03,1.000000,1.0,7.766600e+04,1.882200e+05,2.016000e+03,7.970000e+04,2.468620e+03,14.000000,6.037400e+13
50%,535.000000,2.000000e+00,3.000000e+00,2.000000e+00,1281.000000,1.574000e+03,1.542000e+03,1296.000000,2173.000000,1284.000000,...,1.963000e+03,1.000000,1.0,1.270660e+05,3.211610e+05,2.016000e+03,1.766190e+05,4.007620e+03,14.000000,6.037572e+13
75%,847.500000,3.000000e+00,4.000000e+00,3.000000e+00,1615.000000,2.140000e+03,2.075000e+03,1440.000000,2975.000000,1619.000000,...,1.981000e+03,2.000000,1.0,2.040000e+05,5.140720e+05,2.016000e+03,3.261000e+05,6.230500e+03,15.000000,6.059042e+13
max,8516.000000,3.200000e+01,2.500000e+01,3.200000e+01,31303.000000,9.525760e+05,4.270790e+05,2688.000000,820242.000000,41906.000000,...,2.016000e+03,41.000000,1.0,2.553212e+08,3.196225e+08,2.016000e+03,9.401108e+07,3.823176e+06,99.000000,4.830301e+14


In [70]:
#clean squarefeet columns
sqr_ft_columns = []
for i in clean_df.columns.tolist():
    if 'finishedsquarefeet' in i:
        sqr_ft_columns += [i]
        
clean_df['square_feet'] = clean_df[sqr_ft_columns].mean(axis = 1)
clean_df = clean_df.drop(sqr_ft_columns, axis = 1)

In [82]:
#drop columns with few values
suffecient_columns = clean_df.count(axis = 0)[clean_df.count(axis = 0) > 100_000].index.tolist()
clean_df = clean_df[suffecient_columns]
clean_df.describe()

,bathroomcnt,bedroomcnt,calculatedbathnbr,finishedfloor1squarefeet,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,...,threequarterbathnbr,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,square_feet
count,2.982260e+06,2.982272e+06,2.868061e+06,203758.000000,2.982285e+06,313124.000000,2.868061e+06,891008.000000,891008.00000,2.712511e+06,...,316357.000000,1.981042e+06,2.937384e+06,685676.000000,2.938753e+06,2.950951e+06,2.982284e+06,2.925291e+06,2.962465e+06,2.940120e+06
mean,2.216196e+00,3.093362e+00,2.303826e+00,1379.780460,6.048053e+03,1.168959,2.248089e+00,1.826581,383.15622,2.260376e+04,...,1.010611,1.180303e+00,1.964438e+03,1.400956,1.781429e+05,4.435279e+05,2.015998e+03,2.684558e+05,5.408949e+03,1.819999e+03
std,1.078250e+00,1.271569e+00,1.003243e+00,634.422849,2.024082e+01,0.461855,9.929959e-01,0.608934,246.21942,2.499836e+05,...,0.120602,2.486949e+00,2.364428e+01,0.539056,4.600503e+05,8.163366e+05,6.039949e-02,4.865097e+05,9.675567e+03,1.950845e+03
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,6.037000e+03,1.000000,1.000000e+00,0.000000,0.00000,1.000000e+02,...,1.000000,1.000000e+00,1.801000e+03,1.000000,1.000000e+00,1.000000e+00,2.000000e+03,1.000000e+00,2.400000e-01,1.000000e+00
25%,2.000000e+00,2.000000e+00,2.000000e+00,1010.000000,6.037000e+03,1.000000,2.000000e+00,2.000000,312.00000,5.683000e+03,...,1.000000,1.000000e+00,1.950000e+03,1.000000,7.766600e+04,1.882200e+05,2.016000e+03,7.970000e+04,2.468620e+03,1.210000e+03
50%,2.000000e+00,3.000000e+00,2.000000e+00,1281.000000,6.037000e+03,1.000000,2.000000e+00,2.000000,441.00000,7.000000e+03,...,1.000000,1.000000e+00,1.963000e+03,1.000000,1.270660e+05,3.211610e+05,2.016000e+03,1.766190e+05,4.007620e+03,1.567000e+03
75%,3.000000e+00,4.000000e+00,3.000000e+00,1615.000000,6.059000e+03,1.000000,3.000000e+00,2.000000,494.00000,9.893000e+03,...,1.000000,1.000000e+00,1.981000e+03,2.000000,2.040000e+05,5.140720e+05,2.016000e+03,3.261000e+05,6.230500e+03,2.121000e+03
max,3.200000e+01,2.500000e+01,3.200000e+01,31303.000000,6.111000e+03,9.000000,3.200000e+01,25.000000,7749.00000,3.710005e+08,...,7.000000,9.970000e+02,2.016000e+03,41.000000,2.553212e+08,3.196225e+08,2.016000e+03,9.401108e+07,3.823176e+06,9.525760e+05


In [73]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 26 columns):
bathroomcnt                   float64
bedroomcnt                    float64
calculatedbathnbr             float64
finishedfloor1squarefeet      float64
fips                          float64
fireplacecnt                  float64
fullbathcnt                   float64
garagecarcnt                  float64
garagetotalsqft               float64
latitude                      float64
longitude                     float64
lotsizesquarefeet             float64
poolcnt                       float64
rawcensustractandblock        float64
roomcnt                       float64
threequarterbathnbr           float64
unitcnt                       float64
yearbuilt                     float64
numberofstories               float64
structuretaxvaluedollarcnt    float64
taxvaluedollarcnt             float64
assessmentyear                float64
landtaxvaluedollarcnt         float64
taxamount    

In [ ]:
#drop locations:
clean_df = clean_df.drop(['latitude', 'longitude', 'rawcensustractandblock', 'censustractandblock'], axis = 1)

In [85]:
#drop redundancies
clean_df = clean_df.drop(['calculatedbathnbr', 'finishedfloor1squarefeet', 'roomcnt', 'threequarterbathnbr', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt'], axis = 1)

In [87]:
clean_df.describe()

,bathroomcnt,bedroomcnt,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,lotsizesquarefeet,poolcnt,unitcnt,yearbuilt,numberofstories,assessmentyear,taxamount,square_feet
count,2.982260e+06,2.982272e+06,2.982285e+06,313124.000000,2.868061e+06,891008.000000,891008.00000,2.712511e+06,539632.0,1.981042e+06,2.937384e+06,685676.000000,2.982284e+06,2.962465e+06,2.940120e+06
mean,2.216196e+00,3.093362e+00,6.048053e+03,1.168959,2.248089e+00,1.826581,383.15622,2.260376e+04,1.0,1.180303e+00,1.964438e+03,1.400956,2.015998e+03,5.408949e+03,1.819999e+03
std,1.078250e+00,1.271569e+00,2.024082e+01,0.461855,9.929959e-01,0.608934,246.21942,2.499836e+05,0.0,2.486949e+00,2.364428e+01,0.539056,6.039949e-02,9.675567e+03,1.950845e+03
min,0.000000e+00,0.000000e+00,6.037000e+03,1.000000,1.000000e+00,0.000000,0.00000,1.000000e+02,1.0,1.000000e+00,1.801000e+03,1.000000,2.000000e+03,2.400000e-01,1.000000e+00
25%,2.000000e+00,2.000000e+00,6.037000e+03,1.000000,2.000000e+00,2.000000,312.00000,5.683000e+03,1.0,1.000000e+00,1.950000e+03,1.000000,2.016000e+03,2.468620e+03,1.210000e+03
50%,2.000000e+00,3.000000e+00,6.037000e+03,1.000000,2.000000e+00,2.000000,441.00000,7.000000e+03,1.0,1.000000e+00,1.963000e+03,1.000000,2.016000e+03,4.007620e+03,1.567000e+03
75%,3.000000e+00,4.000000e+00,6.059000e+03,1.000000,3.000000e+00,2.000000,494.00000,9.893000e+03,1.0,1.000000e+00,1.981000e+03,2.000000,2.016000e+03,6.230500e+03,2.121000e+03
max,3.200000e+01,2.500000e+01,6.111000e+03,9.000000,3.200000e+01,25.000000,7749.00000,3.710005e+08,1.0,9.970000e+02,2.016000e+03,41.000000,2.016000e+03,3.823176e+06,9.525760e+05


In [93]:
#change nans in count columns to zeros
clean_df.fireplacecnt.fillna(0, inplace = True)
clean_df.garagecarcnt.fillna(0, inplace = True)
clean_df.garagecarcnt.fillna(0, inplace = True)
clean_df.garagetotalsqft.fillna(0, inplace = True)
clean_df.poolcnt.fillna(0, inplace = True)
clean_df.numberofstories.fillna(1, inplace = True)


In [100]:
clean_df.dropna().count()

bathroomcnt          1911393
bedroomcnt           1911393
fips                 1911393
fireplacecnt         1911393
fullbathcnt          1911393
garagecarcnt         1911393
garagetotalsqft      1911393
lotsizesquarefeet    1911393
poolcnt              1911393
unitcnt              1911393
yearbuilt            1911393
numberofstories      1911393
assessmentyear       1911393
taxamount            1911393
square_feet          1911393
dtype: int64

In [120]:
#need housing prices(target variable)
clean_df = clean_df.join(df.id, how = 'inner').dropna()
clean_df = clean_df.merge(df[['id', 'taxvaluedollarcnt']], on = 'id').dropna()

In [131]:
features = clean_df.columns.tolist()[:-1]
target = [clean_df.columns.tolist()[-1]]
target

['taxvaluedollarcnt']

In [140]:
train, test = ss.split_my_data(clean_df)

In [141]:
scaler, train[features], test[features] = ss.standard_scaler(train[features],test[features])

In [142]:
fe.lasso_cv_coef(train[features], train[target])

bathroomcnt               0.000000
bedroomcnt            -6068.465801
fips                      0.000000
fireplacecnt              0.000000
fullbathcnt            4210.758839
garagecarcnt            916.708150
garagetotalsqft          -0.000000
lotsizesquarefeet        88.729797
poolcnt                2271.733865
unitcnt               -5023.942171
yearbuilt             -7059.887861
numberofstories         522.258172
assessmentyear            0.000000
taxamount            626416.026377
square_feet            2495.152237
id                       -0.000000
dtype: float64